In [3]:
import requests
path = 'http://exercise.kingname.info/exercise_requests_get.html'
html_str = requests.get(path).content.decode()
print(html_str)

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>GET 测试</title>
</head>
<body>
<p>如果你可以看到这一段话，那么说明你已经知道如何通过</p>
<p>requests获得GET方式网页的源代码。</p>
</body>
</html>


In [13]:
pip install bs4

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [8]:
path = 'http://exercise.kingname.info/exercise_requests_post'
data = {'name':'songruochen','password':123456}
html_json = requests.post(path,json = data).content.decode()
print(html_json)

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>POST 测试</title>
</head>
<body>

<p>恭喜恭喜，你现在使用的是POST方式访问本页，通过JSON提交数据，你提交的数据为：</p>
<p>Name: songruochen</p>
<p>Password: 123456</p>

</body>
</html>


In [9]:
path = 'http://exercise.kingname.info/exercise_requests_post'
data = {'name':'songruochen','password':123456}
html_formdata = requests.post(path,data = data).content.decode()
print(html_formdata)

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>POST 测试</title>
</head>
<body>

<p>恭喜恭喜，你现在使用的是POST方式访问本页，通过Formdata提交数据，你提交的数据为：</p>
<p>Name: songruochen</p>
<p>Password: 123456</p>

</body>
</html>


# 结合requests和正则表达式提取网页信息

In [14]:
import requests
import re
path = 'http://exercise.kingname.info/exercise_requests_get.html'
html = requests.get(path).content.decode() # 获得源代码
title = re.search('<title>(.*?)</title>',html,re.S).group(1)
content_list = re.findall('<p>(.*?)</p>',html,re.S)
content_str = ''.join(content_list)
print(f'网页标题为:',title)
print(f'网页内容为:',content_str)


网页标题为: GET 测试
网页内容为: 如果你可以看到这一段话，那么说明你已经知道如何通过requests获得GET方式网页的源代码。


# 小说网站——蛊真人——爬虫开发

In [14]:
import re
import requests 
import csv
import pandas as pd
from bs4 import BeautifulSoup

In [55]:
from bs4 import BeautifulSoup
import requests
import csv
import re

url = 'https://arxiv.org/list/cond-mat.mtrl-sci/recent?skip=100&show=2000'
response = requests.get(url).content.decode()
print(response)

<!DOCTYPE html>
<html lang="en">

<head>  <title>Materials Science  </title>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="apple-touch-icon" sizes="180x180" href="/static/browse/0.3.4/images/icons/apple-touch-icon.png">
  <link rel="icon" type="image/png" sizes="32x32" href="/static/browse/0.3.4/images/icons/favicon-32x32.png">
  <link rel="icon" type="image/png" sizes="16x16" href="/static/browse/0.3.4/images/icons/favicon-16x16.png">
  <link rel="manifest" href="/static/browse/0.3.4/images/icons/site.webmanifest">
  <link rel="mask-icon" href="/static/browse/0.3.4/images/icons/safari-pinned-tab.svg" color="#5bbad5">
  <meta name="msapplication-TileColor" content="#da532c">
  <meta name="theme-color" content="#ffffff">
  <link rel="stylesheet" type="text/css" media="screen" href="/static/browse/0.3.4/css/arXiv.css?v=20241206" />
  <link rel="stylesheet" type="text/css" media="print" href="/static/browse/0.3.4/css/arXiv-print.css?v=20200611" />
  <

我们只想关注化学领域的文献，因此寻找相关的item

In [56]:
papers = re.findall("a name='(.*?)<dt>", response,re.S)
print(len(papers))

79


In [57]:
import pandas as pd
data = {}
title_list = []
author_name = []
doi_list = []
html_list = []
for each in papers:
    # 提取标题
    title_match = re.findall(r"Title:</span>\n          (.*?)\n        </div>", each, re.S)
    if title_match:  # 确保有匹配结果
        title_list.append(title_match[0])  # 取第一个匹配结果

    # 提取作者信息
    author_information = re.findall(r"<div class='list-authors'>(.*?)</div>", each, re.S)
    if author_information:  # 确保有匹配结果
        authors = re.findall(r'<a[^>]*>([^<]+)</a>', author_information[0], re.S)  # 从字符串中提取作者
        author_name.append(", ".join(authors))  # 将作者列表添加到 author_name
    arvix_doi_match = re.findall(r"arXiv:(.*?)\n",each,re.S)
    if arvix_doi_match:
        doi_list.append(arvix_doi_match[0])
    html_match = re.findall(r'<a href="(https://arxiv\.org/html/.*?)"', each,re.S)
    #print(html_match)
    if html_match:
        html_list.append(html_match[0])
    else:
        html_list.append("")
# 将结果存入字典
data['title'] = title_list
data['author'] = author_name
data['doi'] = doi_list
data['html'] = html_list
df = pd.DataFrame(data)
# 打印结果
df_reset = df.reset_index(drop = True)
df_reset

,title,author,doi,html
0,Identification of Polytypism and Their Disloca...,"Xin Zhou, Tobias Dierke, Mingjian Wu, Shengbo ...",2502.11977,
1,Weyl and Dirac Semimetals for Thermoelectric A...,"Saurabh Singh, Sarmistha Das, Shiv Kumar",2502.11963,
2,Uncovering the atomic structure of substitutio...,"David Lamprecht, Anna Benzer, Manuel Längle, M...",2502.11912,https://arxiv.org/html/2502.11912v1
3,Evolution of radiation-induced damage in nucle...,"Magdalena Wilczopolska, Kinga Suchorab, Magdal...",2502.11851,
4,Exploring Novel 2D Analogues of Goldene: Elect...,"Emanuel J. A. dos Santos, Rodrigo A. F. Alves,...",2502.11810,https://arxiv.org/html/2502.11810v1
...,...,...,...,...
74,Impurity-induced moment freezing in NaFe$_x$Ru...,"Alon Hendler Avidor, Brenden R. Ortiz, Paul M....",2502.10343,https://arxiv.org/html/2502.10343v1
75,Agentic End-to-End De Novo Protein Design for ...,"Bo Ni, Markus J. Buehler",2502.10173,
76,Light-induced dissipationless states in magnet...,"Mohammad Shafiei, Milorad V. Milošević",2502.10164,https://arxiv.org/html/2502.10164v1
77,Fractional-flux oscillations of Josephson crit...,"M. Donaire, A. Cano",2502.09964,https://arxiv.org/html/2502.09964v1


现在尝试封装这些功能为一体

In [58]:
import re
import requests 
import csv
import pandas as pd
from bs4 import BeautifulSoup

In [62]:
import re
import requests 
import csv
import pandas as pd

urls = ['https://arxiv.org/list/cond-mat.mtrl-sci/recent?skip=0&show=100','https://arxiv.org/list/cond-mat.mtrl-sci/recent?skip=100&show=100']
data = {}
title_list = []
author_name = []
doi_list = []
html_list = []
Id_list = []
for url in urls:
    reponse = requests.get(url).content.decode()
    papers = re.findall("a name='(.*?)<dt>", response,re.S)
    for each in papers:
    # 读取Id
        Id_match = re.findall(r"<a name='item(.*?)'>",each,re.S)
        if Id_match:
            Id_list.append(Id)
        else:
            Id_list.append("")
    # 提取标题
        title_match = re.findall(r"Title:</span>\n          (.*?)\n        </div>", each, re.S)
        if title_match:  # 确保有匹配结果
            title_list.append(title_match[0])  # 取第一个匹配结果
        else:
            title_list.append("")
    # 提取作者信息
        author_information = re.findall(r"<div class='list-authors'>(.*?)</div>", each, re.S)
        if author_information:  # 确保有匹配结果
            authors = re.findall(r'<a[^>]*>([^<]+)</a>', author_information[0], re.S)  # 从字符串中提取作者
            author_name.append(", ".join(authors))  # 将作者列表添加到 author_name
        else:
            author_name.append("")
        arvix_doi_match = re.findall(r"arXiv:(.*?)\n",each,re.S)
    # 读取arvix doi
        if arvix_doi_match:
            doi_list.append(arvix_doi_match[0])
        else:
            doi_list.append("")
    # 读取文献的html
        html_match = re.findall(r'<a href="(https://arxiv\.org/html/.*?)"', each,re.S)
        if html_match:
            html_list.append(html_match[0])
        else:
            html_list.append("")
    data['title'] = title_list
    data['author'] = author_name
    data['doi'] = doi_list
    data['html'] = html_list
    data['Id'] = Id_list
df = pd.DataFrame(data)
df

,title,author,doi,html,Id
0,Identification of Polytypism and Their Disloca...,"Xin Zhou, Tobias Dierke, Mingjian Wu, Shengbo ...",2502.11977,,
1,Weyl and Dirac Semimetals for Thermoelectric A...,"Saurabh Singh, Sarmistha Das, Shiv Kumar",2502.11963,,
2,Uncovering the atomic structure of substitutio...,"David Lamprecht, Anna Benzer, Manuel Längle, M...",2502.11912,https://arxiv.org/html/2502.11912v1,
3,Evolution of radiation-induced damage in nucle...,"Magdalena Wilczopolska, Kinga Suchorab, Magdal...",2502.11851,,
4,Exploring Novel 2D Analogues of Goldene: Elect...,"Emanuel J. A. dos Santos, Rodrigo A. F. Alves,...",2502.11810,https://arxiv.org/html/2502.11810v1,
...,...,...,...,...,...
153,Impurity-induced moment freezing in NaFe$_x$Ru...,"Alon Hendler Avidor, Brenden R. Ortiz, Paul M....",2502.10343,https://arxiv.org/html/2502.10343v1,
154,Agentic End-to-End De Novo Protein Design for ...,"Bo Ni, Markus J. Buehler",2502.10173,,
155,Light-induced dissipationless states in magnet...,"Mohammad Shafiei, Milorad V. Milošević",2502.10164,https://arxiv.org/html/2502.10164v1,
156,Fractional-flux oscillations of Josephson crit...,"M. Donaire, A. Cano",2502.09964,https://arxiv.org/html/2502.09964v1,


还是得要模块化

In [110]:
# 全局列表，用于存储提取的数据
Id_list = []
title_list = []
author_list = []
doi_list = []
html_list = []
# 定义提取函数
def get_id(html):
    Id_match = re.findall(r"<a name='item(.*?)'>",html,re.S)
    if Id_match:
        Id_list.append(Id_match[0])
    else:
        Id_list.append("")

def get_title(html):
    title_match = re.findall(r"Title:</span>\n          (.*?)\n        </div>", html, re.S)
    if title_match:  # 确保有匹配结果
        title_list.append(title_match[0])  # 取第一个匹配结果
    else:
        title_list.append("")
  
def get_author(html):
    author_information = re.findall(r"<div class='list-authors'>(.*?)</div>", html, re.S)#找到大字符块
    if author_information:  # 确保有匹配结果
        authors = re.findall(r'<a[^>]*>([^<]+)</a>', author_information[0], re.S)  # 从字符串中提取作者
        author_list.append(", ".join(authors))  # 将作者列表添加到 author_name
    else:
        author_list.append("")
   
def get_doi(html):
    arvix_doi_match = re.findall(r"arXiv:(.*?)\n",html,re.S)
    if arvix_doi_match:
        doi_list.append(arvix_doi_match[0])
    else:
        doi_list.append("")
        
def get_html(html):
    html_match = re.findall(r'<a href="(https://arxiv\.org/html/.*?)"', html,re.S)
    if html_match:
        html_list.append(html_match[0])
    else:
        html_list.append("")
    

urls = ['https://arxiv.org/list/cond-mat.mtrl-sci/recent?skip=0&show=100','https://arxiv.org/list/cond-mat.mtrl-sci/recent?skip=100&show=100']
# 遍历 papers，提取数据
for url in urls:
    response = requests.get(url).content.decode()
    papers = re.findall(r" <dt>(.*?)</dd>", response, re.S)
    for each in papers:
        get_id(each)
        get_title(each)
        get_author(each)
        get_doi(each)
        get_html(each)

# 将结果存入 DataFrame
data = {
    'ID': Id_list,
    'Title': title_list,
    'Author': author_list,
    'DOI': doi_list,
    'HTML': html_list
}
df = pd.DataFrame(data)
df['ID']


0        1
1        2
2        3
3        4
4        5
      ... 
175    176
176    177
177    178
178    179
179    180
Name: ID, Length: 180, dtype: object

最终完整获得 2页共180篇文献的信息

In [111]:
df.to_csv('arvix_ouput_2.csv',index=False)